In [3]:
from product_sub.infrastructure.dataset_builder import DatasetBuilder
from product_sub.domain.data_cleaning import NumImputer, CatImputer
from product_sub.domain.feature_selector import FeatureSelector
import product_sub.settings as stg

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.pipeline import FeatureUnion, Pipeline,  make_pipeline, make_union
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.compose import make_column_selector as selector
from sklearn.compose import ColumnTransformer

import numpy as np
import pandas as pd


RANDOM_STATE=42

### Create Dataset

In [2]:
dataset_merged = DatasetBuilder(
    filename_bank="data.csv", filename_socio="socio_eco.csv"
).create_dataset()
X = dataset_merged.drop(columns=stg.COL_RAW_SUBSCRIPTION)
y = dataset_merged[stg.COL_RAW_SUBSCRIPTION].values

### Pipeline

In [ ]:
numeric_transformer = Pipeline_sickit(steps=[
    ('num_imputer', NumImputer()),
    ('scaler', StandardScaler())])
cat_imputer = CatImputer(['JOB_TYPE', 'EDUCATION', 'STATUS'])
categorical_transformer = Pipeline_sickit(steps=[
    ('cat_imputer', cat_imputer ),
    ('encoder', OneHotEncoder())
])

preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, selector(dtype_exclude="category")),
    ('cat', categorical_transformer, selector(dtype_include="category"))
])
clf = Pipeline_sickit(steps=[ ('preprocessor' , preprocessor),
                           ('classifier',  GradientBoostingClassifier()) ])